# Neo4j Official MCP Agent (stdio)

This notebook demonstrates how to use the **Official Neo4j MCP Server** (`mcp-neo4j-cypher`) via the standard **stdio** transport.

This simulates exactly how an AI Desktop Client (like Claude Desktop or a local AgentCore runner) would connect to the Neo4j MCP server running as a subprocess.

We will use the **Neo4j Demo Database** (ReadOnly).


In [ ]:
!pip install mcp mcp-neo4j-cypher neo4j

## 1. Environment Configuration

We need to set the environment variables that the subprocess will inherit to connect to Neo4j.

In [ ]:
import os

# Public Demo Database Credentials
os.environ["NEO4J_URI"] = "neo4j+s://demo.neo4jlabs.com:7687"
os.environ["NEO4J_USERNAME"] = "companies"
os.environ["NEO4J_PASSWORD"] = "companies"
os.environ["NEO4J_DATABASE"] = "companies"

print("Environment variables set for subprocess.")

## 2. Connect to MCP Server via Stdio

We will use the `mcp` python client to launch `mcp-neo4j-cypher` as a subprocess and communicate over stdio.
The server exposes standard tools like `read-neo4j-cypher` functionality.

In [ ]:
import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

# Define how to run the server
server_params = StdioServerParameters(
    command="mcp-neo4j-cypher", # This executable is installed by pip env
    args=[], # No args needed as config is in env vars
    env=os.environ.copy()
)

async def run_mcp_session():
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            # 1. Initialize the session
            await session.initialize()
            
            # 2. List available tools to confirm connection
            tools = await session.list_tools()
            print(f"Connected! Found {len(tools.tools)} tools:")
            for tool in tools.tools:
                print(f" - {tool.name}: {tool.description[:50]}...")
            
            # 3. Simulate an Agent Request
            # Agent wants to find generic info about Google
            cypher_query = "MATCH (o:Organization {name: 'Google'}) RETURN o.name, o.url LIMIT 1"
            
            print("\nExecuting Cypher via MCP...")
            # Note: The tool name is specific to the server implementation
            # For mcp-neo4j-cypher, the read tool is likely 'read-neo4j-cypher' (dashes) or similar.
            # Let's dynamically find the read tool.
            read_tool = next((t for t in tools.tools if "read" in t.name or "cypher" in t.name), None)
            
            if read_tool:
                result = await session.call_tool(
                    read_tool.name,
                    arguments={"query": cypher_query}
                )
                print(f"Result: {result.content[0].text}")
            else:
                print("Could not find a Cypher read tool.")

# Run the async function
# In Jupyter, we can just await it directly if top-level await is enabled, 
# or use asyncio.run() if not in a running loop (Jupyter has a loop).
# For compatibility, we'll just check/run.
try:
    await run_mcp_session()
except RuntimeError:
    # If loop acts up (rare in modern Jupyter)
    asyncio.run(run_mcp_session())

## 3. Deployment Context

To use this with **AWS AgentCore**:

1.  **Docker**: You would package `mcp-neo4j-cypher` into a Docker container and run it on ECS/Fargate.
2.  **AgentCore**: Configure AgentCore to connect to that container (Gateway pattern) or use the `mcp-neo4j` library in a Lambda function (as shown in the other notebook for custom tools, but importing the official server classes instead).